In [ ]:
from itertools import product
import numpy as np
import pandas as pd
from astroquery.vizier import Vizier
import matplotlib.pyplot as plt
import astropy as ap
import george
from george import kernels
import pymc3 as pm
import theano
import theano.tensor as tt
import sklearn
from sklearn.neighbors import KernelDensity as KD
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, KernelPCA
import corner

In [ ]:
Vizier = Vizier(row_limit=20000)

In [ ]:
catalog = Vizier.get_catalogs("J/A+A/618/A93")

clucata = catalog[1]

In [ ]:
newc = clucata.group_by('Cluster')

maxcluster = np.argmax(newc.groups.indices[1:]-newc.groups.indices[:-1])
bigcluster = newc.groups[maxcluster+6]
print(bigcluster.colnames)
newc.groups[maxcluster+6]['Cluster'][0]

Good `'Cluster'`s to choose from: Alessi\_24, ASCC_99, Alessi\_12

In [ ]:
cutcluster = bigcluster[bigcluster['PMemb']>.8]
cutcluster = cutcluster[~np.isnan(cutcluster["BP-RP"])]
#We should cut by lines away from main sequence
cutcluster = cutcluster[~np.logical_and(cutcluster["BP-RP"]>1.0,cutcluster["Gmag"]<10.)]
cutcluster = cutcluster[~np.logical_and(cutcluster["BP-RP"]<.7, cutcluster["Gmag"]>13.8)]
plt.plot(cutcluster['RA_ICRS'],cutcluster['DE_ICRS'],'+')
plt.title('angular coordinates of '+cutcluster['Cluster'][0])

In [ ]:
plt.hist(cutcluster['PMemb'])
plt.xlabel('PMemb')
plt.title('cluster membership probability of '+cutcluster['Cluster'][0])

In [ ]:
plt.plot(cutcluster['BP-RP'],cutcluster['Gmag'], '+')
plt.ylim(19, 7)
plt.xlabel('BP-RP')
plt.ylabel('Gmag')
plt.title('color-magnitude diagram of '+cutcluster['Cluster'][0])

PCA - diagram, inverse of variance as metric -> sqrt(thing)= transformation, apply ->PCA(test) -> KDE -> inverse transform both L and R on Kernel Widths

LLE? -> Local linear embedding -> for non MS

In [ ]:
remove outliers before PCA

In [ ]:
X = np.asarray(np.vstack((cutcluster['BP-RP'], cutcluster['Gmag']))).T

In [ ]:
kpca = KernelPCA(kernel="linear", n_components=5, fit_inverse_transform=True, gamma=10)

In [ ]:
X_kpca = kpca.fit_transform(X)
X_back = kpca.inverse_transform(X_kpca)
pca = PCA()
X_pca = pca.fit_transform(X)

In [ ]:
print(X_kpca.shape)
print(X_back.shape)
print(X_pca.shape)

In [ ]:
X1, X2 = np.meshgrid(np.linspace(0, 3, 50), np.linspace(18, 7, 50))
X_grid = np.array([np.ravel(X1), np.ravel(X2)]).T
X_transform = kpca.transform(X)
Z_grid = kpca.transform(X_grid)[:, 0].reshape(X1.shape)
plt.contour(X1, X2, Z_grid);

In [ ]:
X_transform = kpca.transform(X)
X_transform.shape
plt.scatter(X_transform[:,0], X_transform[:,1])

In [ ]:
v̂ = kpca.alphas_
plt.scatter(v̂[:,0],v̂[:,1])

In [ ]:
plt.hist(kpca.alphas_[:,1])

In [ ]:
corner.corner(X_transform);

In [ ]:
plt.scatter(X_back[:, 0], X_back[:, 1], c="red",
            s=20, edgecolor='k')

In [ ]:
cmd = np.asarray(np.vstack((cutcluster['BP-RP'], cutcluster['Gmag']))).T
print((np.min(cutcluster['BP-RP']), np.max(cutcluster['BP-RP']), np.min(cutcluster['Gmag']), np.max(cutcluster['Gmag'])))

In [ ]:
corner.corner(cmd)

In [ ]:
# I couldn't immediately find a KDE code that enabled different bandwidth in each dimension
# which we want because the errors in color are much greater than the errors in magnitude
params = {'bandwidth': np.logspace(-2, 0, 200)}
grid = GridSearchCV(KD(kernel='exponential'), params, cv=5)
grid.fit(cmd)

print("best bandwidth: {0}".format(grid.best_estimator_.bandwidth))
# first attempt obviously too fine a bandwidth because it allows for double stars
# we could fix it here or say this is just what the data is and fit an HRD model that doesn't permit those
# so now the data is the KDE evaluated on a grid

kde = grid.best_estimator_.fit(cmd)
eval_where = np.array(list(product(np.linspace(0., 2.7, 50), np.linspace(18., 7., 50))))
log_dens = kde.score_samples(eval_where)

plt.imshow(np.flip(np.exp(log_dens.reshape(50, 50).T), axis=0), extent=[0., 2.7, 18., 7.], aspect=0.25)
plt.scatter(cutcluster['BP-RP'], cutcluster['Gmag'], marker='.', color='r', s=1)

In [ ]:
x = eval_where
y = log_dens
y.shape
x.shape

In [ ]:
kernel = 5*kernels.Matern52Kernel(5., ndim=2)
gp = george.GP(kernel, mean=np.mean(y), fit_mean=True)

In [ ]:
gp.compute(x)

test_x = np.array(list(product(np.linspace(0, 3, 100),np.linspace(7,18,100))))

In [ ]:
pred, pred_var = gp.predict(y, test_x)

In [ ]:
plt.imshow(np.exp(pred.reshape(100, 100).T), extent=[0., 3, 18., 7.], aspect=0.25)

In [ ]:
import scipy.optimize as op

# Define the objective function (negative log-likelihood in this case).
def nll(p):
    gp.set_parameter_vector(p)
    ll = gp.log_likelihood(y, quiet=True)
    return -ll if np.isfinite(ll) else 1e25

# And the gradient of the objective function.
def grad_nll(p):
    gp.set_parameter_vector(p)
    return -gp.grad_log_likelihood(y, quiet=True)

# You need to compute the GP once before starting the optimization.
gp.compute(x)

# Print the initial ln-likelihood.
print(gp.log_likelihood(y))

# Run the optimization routine.
p0 = gp.get_parameter_vector()
results = op.minimize(nll, p0, jac=grad_nll, method="L-BFGS-B")

# Update the kernel and print the final log-likelihood.
gp.set_parameter_vector(results.x)
print(gp.log_likelihood(y))



In [ ]:
pred, pred_var = gp.predict(y, test_x)


plt.scatter(cutcluster['BP-RP'], cutcluster['Gmag'], marker='.', color='r', s=1)
plt.imshow(np.exp(pred.reshape(100, 100).T), extent=[0., 3, 18., 7.], aspect=0.25)